# BERT Embeddings Clustering Analysis

This notebook performs clustering analysis on H&M product articles using pre-computed BERT embeddings.
The BERT embeddings capture semantic information from product descriptions, enabling meaningful product groupings.


In [ ]:
import sys
sys.path.append('../../')

import polars as pl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist

try:
    import umap
    UMAP_AVAILABLE = True
except ImportError:
    UMAP_AVAILABLE = False
    print("UMAP not available. Install with: pip install umap-learn")

try:
    import hdbscan
    HDBSCAN_AVAILABLE = True
except ImportError:
    HDBSCAN_AVAILABLE = False
    print("HDBSCAN not available. Install with: pip install hdbscan")

import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
sns.set_palette("husl")

## 1. Data Loading and Exploration


In [ ]:
# Load BERT embeddings
print("Loading BERT embeddings...")
bert_df = pl.read_parquet('../../data/features/bert/bert_embeddings.parquet')

print(f"Dataset shape: {bert_df.shape}")
print(f"Memory usage: {bert_df.estimated_size() / 1024**2:.1f} MB")
print(f"Columns: {len(bert_df.columns)} (article_id + {len(bert_df.columns)-1} BERT features)")

# Display basic statistics
bert_df.head()

In [ ]:
# Extract embeddings matrix and article IDs
article_ids = bert_df.get_column('article_id').to_numpy()
embedding_cols = [col for col in bert_df.columns if col.startswith('bert_')]
embeddings = bert_df.select(embedding_cols).to_numpy()

print(f"Embeddings shape: {embeddings.shape}")
print(f"Embedding dimensions: {len(embedding_cols)}")
print(f"Data type: {embeddings.dtype}")

# Check for missing values
missing_count = np.isnan(embeddings).sum()
print(f"Missing values: {missing_count}")

# Basic statistics
print(f"\nEmbedding statistics:")
print(f"Mean: {embeddings.mean():.4f}")
print(f"Std: {embeddings.std():.4f}")
print(f"Min: {embeddings.min():.4f}")
print(f"Max: {embeddings.max():.4f}")

## 2. Dimensionality Reduction for Visualisation


In [ ]:
# PCA for initial dimensionality reduction
print("Performing PCA...")
pca = PCA(n_components=50, random_state=42)
embeddings_pca = pca.fit_transform(embeddings)

print(f"PCA explained variance ratio: {pca.explained_variance_ratio_[:10]}")
print(f"Cumulative explained variance (50 components): {pca.explained_variance_ratio_.sum():.3f}")

# Plot explained variance
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(range(1, 21), pca.explained_variance_ratio_[:20], 'bo-')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('PCA Explained Variance (First 20 Components)')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(range(1, 51), np.cumsum(pca.explained_variance_ratio_), 'ro-')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance')
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# t-SNE for 2D visualisation (using PCA-reduced data for speed)
print("Performing t-SNE...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
embeddings_tsne = tsne.fit_transform(embeddings_pca)

print(f"t-SNE shape: {embeddings_tsne.shape}")

# Plot t-SNE
plt.figure(figsize=(10, 8))
plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1], alpha=0.6, s=1)
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.title('t-SNE Visualisation of BERT Embeddings')
plt.legend()
plt.show()

In [ ]:
# UMAP if available
if UMAP_AVAILABLE:
    print("Performing UMAP...")
    umap_reducer = umap.UMAP(n_components=2, random_state=42, n_neighbors=15, min_dist=0.1)
    embeddings_umap = umap_reducer.fit_transform(embeddings_pca)
    
    plt.figure(figsize=(10, 8))
    plt.scatter(embeddings_umap[:, 0], embeddings_umap[:, 1], alpha=0.6, s=1)
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')
    plt.title('UMAP Visualisation of BERT Embeddings')
    plt.show()
else:
    embeddings_umap = None
    print("Skipping UMAP - not available")

## 3. K-means Clustering


In [ ]:
# Determine optimal number of clusters using elbow method
print("Finding optimal number of clusters...")
k_range = range(2, 50)
inertias = []
silhouette_scores = []
calinski_scores = []
davies_bouldin_scores = []

# Use PCA-reduced data for faster computation
X_clustering = embeddings_pca

for k in k_range:
    print(f"Testing k={k}...")
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_clustering)
    
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_clustering, labels))
    calinski_scores.append(calinski_harabasz_score(X_clustering, labels))
    davies_bouldin_scores.append(davies_bouldin_score(X_clustering, labels))

# Plot clustering metrics
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

axes[0, 0].plot(k_range, inertias, 'bo-')
axes[0, 0].set_xlabel('Number of Clusters (k)')
axes[0, 0].set_ylabel('Inertia')
axes[0, 0].set_title('Elbow Method')
axes[0, 0].grid(True)

axes[0, 1].plot(k_range, silhouette_scores, 'ro-')
axes[0, 1].set_xlabel('Number of Clusters (k)')
axes[0, 1].set_ylabel('Silhouette Score')
axes[0, 1].set_title('Silhouette Score')
axes[0, 1].grid(True)

axes[1, 0].plot(k_range, calinski_scores, 'go-')
axes[1, 0].set_xlabel('Number of Clusters (k)')
axes[1, 0].set_ylabel('Calinski-Harabasz Score')
axes[1, 0].set_title('Calinski-Harabasz Score')
axes[1, 0].grid(True)

axes[1, 1].plot(k_range, davies_bouldin_scores, 'mo-')
axes[1, 1].set_xlabel('Number of Clusters (k)')
axes[1, 1].set_ylabel('Davies-Bouldin Score')
axes[1, 1].set_title('Davies-Bouldin Score (Lower is Better)')
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

# Find optimal k based on silhouette score
optimal_k = k_range[np.argmax(silhouette_scores)]
print(f"\nOptimal k based on silhouette score: {optimal_k}")
print(f"Best silhouette score: {max(silhouette_scores):.3f}")

In [ ]:
# Perform final K-means clustering with optimal k
optimal_k = 32
print(f"Performing final K-means clustering with k={optimal_k}...")
final_kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=20)
kmeans_labels = final_kmeans.fit_predict(X_clustering)

print(f"Cluster distribution:")
unique, counts = np.unique(kmeans_labels, return_counts=True)
for cluster, count in zip(unique, counts):
    print(f"Cluster {cluster}: {count} articles ({count/len(kmeans_labels)*100:.1f}%)")

# Calculate final metrics
final_silhouette = silhouette_score(X_clustering, kmeans_labels)
final_calinski = calinski_harabasz_score(X_clustering, kmeans_labels)
final_davies_bouldin = davies_bouldin_score(X_clustering, kmeans_labels)

print(f"\nFinal clustering metrics:")
print(f"Silhouette Score: {final_silhouette:.3f}")
print(f"Calinski-Harabasz Score: {final_calinski:.3f}")
print(f"Davies-Bouldin Score: {final_davies_bouldin:.3f}")

## 4. Visualise K-means Results


In [ ]:
# Visualise K-means clusters on t-SNE
plt.figure(figsize=(12, 10))
scatter = plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1], 
                     c=kmeans_labels, cmap='tab10', alpha=0.7, s=2)
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.title(f'K-means Clustering (k={optimal_k}) - t-SNE Visualisation')
plt.colorbar(scatter, label='Cluster')
plt.show()

# Interactive plotly visualisation
fig = px.scatter(x=embeddings_tsne[:, 0], y=embeddings_tsne[:, 1], 
                color=kmeans_labels.astype(str),
                hover_data={'article_id': article_ids},
                title=f'K-means Clustering (k={optimal_k}) - Interactive t-SNE',
                labels={'x': 't-SNE 1', 'y': 't-SNE 2', 'color': 'Cluster'})
fig.update_traces(marker=dict(size=3, opacity=0.7))
fig.show()

# Save interactive plot
fig.write_html('../../results/clustering/bert_kmeans_tsne_interactive.html')
print("Interactive plot saved to results/clustering/bert_kmeans_tsne_interactive.html")

In [ ]:
# UMAP visualisation if available
if UMAP_AVAILABLE and embeddings_umap is not None:
    plt.figure(figsize=(12, 10))
    scatter = plt.scatter(embeddings_umap[:, 0], embeddings_umap[:, 1], 
                         c=kmeans_labels, cmap='tab10', alpha=0.7, s=2)
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')
    plt.title(f'K-means Clustering (k={optimal_k}) - UMAP Visualisation')
    plt.colorbar(scatter, label='Cluster')
    plt.show()
    
    # Interactive UMAP
    fig = px.scatter(x=embeddings_umap[:, 0], y=embeddings_umap[:, 1], 
                    color=kmeans_labels.astype(str),
                    hover_data={'article_id': article_ids},
                    title=f'K-means Clustering (k={optimal_k}) - Interactive UMAP',
                    labels={'x': 'UMAP 1', 'y': 'UMAP 2', 'color': 'Cluster'})
    fig.update_traces(marker=dict(size=3, opacity=0.7))
    fig.show()
    
    fig.write_html('../../results/clustering/bert_kmeans_umap_interactive.html')
    print("UMAP interactive plot saved to results/clustering/bert_kmeans_umap_interactive.html")

## 5. Hierarchical Clustering


In [ ]:
# Hierarchical clustering with a sample for computational efficiency
sample_size = min(5000, len(embeddings_pca))  # Use subset for dendrogram
sample_idx = np.random.choice(len(embeddings_pca), sample_size, replace=False)
X_sample = embeddings_pca[sample_idx]

print(f"Performing hierarchical clustering on sample of {sample_size} articles...")

# Compute linkage matrix
linkage_matrix = linkage(X_sample, method='ward')

# Plot dendrogram
plt.figure(figsize=(15, 8))
dendrogram(linkage_matrix, truncate_mode='level', p=6)
plt.xlabel('Sample Index or Cluster Size')
plt.ylabel('Distance')
plt.title('Hierarchical Clustering Dendrogram (Sample)')
plt.show()

In [ ]:
# Perform hierarchical clustering on full dataset
print(f"Performing hierarchical clustering on full dataset with {optimal_k} clusters...")
hierarchical = AgglomerativeClustering(n_clusters=optimal_k, linkage='ward')
hierarchical_labels = hierarchical.fit_predict(X_clustering)

print(f"Hierarchical cluster distribution:")
unique, counts = np.unique(hierarchical_labels, return_counts=True)
for cluster, count in zip(unique, counts):
    print(f"Cluster {cluster}: {count} articles ({count/len(hierarchical_labels)*100:.1f}%)")

# Calculate metrics
hier_silhouette = silhouette_score(X_clustering, hierarchical_labels)
hier_calinski = calinski_harabasz_score(X_clustering, hierarchical_labels)
hier_davies_bouldin = davies_bouldin_score(X_clustering, hierarchical_labels)

print(f"\nHierarchical clustering metrics:")
print(f"Silhouette Score: {hier_silhouette:.3f}")
print(f"Calinski-Harabasz Score: {hier_calinski:.3f}")
print(f"Davies-Bouldin Score: {hier_davies_bouldin:.3f}")

In [ ]:
# Visualise hierarchical clustering results
plt.figure(figsize=(12, 10))
scatter = plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1], 
                     c=hierarchical_labels, cmap='tab10', alpha=0.7, s=2)
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.title(f'Hierarchical Clustering ({optimal_k} clusters) - t-SNE Visualisation')
plt.colorbar(scatter, label='Cluster')
plt.show()

# Interactive plot
fig = px.scatter(x=embeddings_tsne[:, 0], y=embeddings_tsne[:, 1], 
                color=hierarchical_labels.astype(str),
                hover_data={'article_id': article_ids},
                title=f'Hierarchical Clustering ({optimal_k} clusters) - Interactive t-SNE',
                labels={'x': 't-SNE 1', 'y': 't-SNE 2', 'color': 'Cluster'})
fig.update_traces(marker=dict(size=3, opacity=0.7))
fig.show()

fig.write_html('../../results/clustering/bert_hierarchical_tsne_interactive.html')
print("Hierarchical clustering interactive plot saved")

## 6. HDBSCAN Clustering


In [ ]:
if HDBSCAN_AVAILABLE:
    print("Performing HDBSCAN clustering...")
    
    # Try different min_cluster_size values
    min_sizes = [50, 100, 200, 500]
    hdbscan_results = []
    
    for min_size in min_sizes:
        clusterer = hdbscan.HDBSCAN(min_cluster_size=min_size, min_samples=10)
        cluster_labels = clusterer.fit_predict(X_clustering)
        
        n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
        n_noise = list(cluster_labels).count(-1)
        
        if n_clusters > 1:
            silhouette = silhouette_score(X_clustering[cluster_labels != -1], 
                                        cluster_labels[cluster_labels != -1])
        else:
            silhouette = -1
        
        hdbscan_results.append({
            'min_size': min_size,
            'n_clusters': n_clusters,
            'n_noise': n_noise,
            'silhouette': silhouette,
            'labels': cluster_labels
        })
        
        print(f"Min size {min_size}: {n_clusters} clusters, {n_noise} noise points, silhouette: {silhouette:.3f}")
    
    # Choose best HDBSCAN result
    best_hdbscan = max(hdbscan_results, key=lambda x: x['silhouette'] if x['silhouette'] > 0 else -1)
    hdbscan_labels = best_hdbscan['labels']
    
    print(f"\nBest HDBSCAN result:")
    print(f"Min cluster size: {best_hdbscan['min_size']}")
    print(f"Number of clusters: {best_hdbscan['n_clusters']}")
    print(f"Noise points: {best_hdbscan['n_noise']} ({best_hdbscan['n_noise']/len(hdbscan_labels)*100:.1f}%)")
    print(f"Silhouette score: {best_hdbscan['silhouette']:.3f}")
    
else:
    print("HDBSCAN not available - skipping")
    hdbscan_labels = None

In [ ]:
if HDBSCAN_AVAILABLE and hdbscan_labels is not None:
    # Visualise HDBSCAN results
    plt.figure(figsize=(12, 10))
    scatter = plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1], 
                         c=hdbscan_labels, cmap='tab10', alpha=0.7, s=2)
    plt.xlabel('t-SNE 1')
    plt.ylabel('t-SNE 2')
    plt.title(f'HDBSCAN Clustering - t-SNE Visualisation')
    plt.colorbar(scatter, label='Cluster')
    plt.show()
    
    # Interactive plot
    fig = px.scatter(x=embeddings_tsne[:, 0], y=embeddings_tsne[:, 1], 
                    color=hdbscan_labels.astype(str),
                    hover_data={'article_id': article_ids},
                    title='HDBSCAN Clustering - Interactive t-SNE',
                    labels={'x': 't-SNE 1', 'y': 't-SNE 2', 'color': 'Cluster'})
    fig.update_traces(marker=dict(size=3, opacity=0.7))
    fig.show()
    
    fig.write_html('../../results/clustering/bert_hdbscan_tsne_interactive.html')
    print("HDBSCAN interactive plot saved")

## 7. Clustering Comparison


In [ ]:
# Compare different clustering methods
comparison_data = {
    'Method': ['K-means', 'Hierarchical'],
    'Silhouette Score': [final_silhouette, hier_silhouette],
    'Calinski-Harabasz Score': [final_calinski, hier_calinski],
    'Davies-Bouldin Score': [final_davies_bouldin, hier_davies_bouldin]
}

if HDBSCAN_AVAILABLE and hdbscan_labels is not None:
    comparison_data['Method'].append('HDBSCAN')
    comparison_data['Silhouette Score'].append(best_hdbscan['silhouette'])
    comparison_data['Calinski-Harabasz Score'].append('N/A')
    comparison_data['Davies-Bouldin Score'].append('N/A')

comparison_df = pd.DataFrame(comparison_data)
print("Clustering Method Comparison:")
print(comparison_df.to_string(index=False))

# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

methods = comparison_df['Method'].tolist()
silhouette_vals = [x for x in comparison_df['Silhouette Score'] if isinstance(x, float)]
methods_numeric = [m for i, m in enumerate(methods) if isinstance(comparison_df.iloc[i]['Silhouette Score'], float)]

axes[0].bar(methods_numeric, silhouette_vals)
axes[0].set_title('Silhouette Score Comparison')
axes[0].set_ylabel('Score')

calinski_vals = [x for x in comparison_df['Calinski-Harabasz Score'] if isinstance(x, (int, float))]
methods_calinski = [m for i, m in enumerate(methods) if isinstance(comparison_df.iloc[i]['Calinski-Harabasz Score'], (int, float))]

if calinski_vals:
    axes[1].bar(methods_calinski, calinski_vals)
axes[1].set_title('Calinski-Harabasz Score Comparison')
axes[1].set_ylabel('Score')

davies_vals = [x for x in comparison_df['Davies-Bouldin Score'] if isinstance(x, (int, float))]
methods_davies = [m for i, m in enumerate(methods) if isinstance(comparison_df.iloc[i]['Davies-Bouldin Score'], (int, float))]

if davies_vals:
    axes[2].bar(methods_davies, davies_vals)
axes[2].set_title('Davies-Bouldin Score Comparison (Lower is Better)')
axes[2].set_ylabel('Score')

plt.tight_layout()
plt.show()

## 8. Save Results


In [ ]:
# Create results dataframe
results_df = pl.DataFrame({
    'article_id': article_ids,
    'kmeans_cluster': kmeans_labels,
    'hierarchical_cluster': hierarchical_labels,
    'tsne_1': embeddings_tsne[:, 0],
    'tsne_2': embeddings_tsne[:, 1]
})

if HDBSCAN_AVAILABLE and hdbscan_labels is not None:
    results_df = results_df.with_columns(
        pl.Series('hdbscan_cluster', hdbscan_labels)
    )

if UMAP_AVAILABLE and embeddings_umap is not None:
    results_df = results_df.with_columns([
        pl.Series('umap_1', embeddings_umap[:, 0]),
        pl.Series('umap_2', embeddings_umap[:, 1])
    ])

# Save results
results_df.write_parquet('../../results/clustering/bert_embeddings_clustering_results.parquet')
results_df.write_csv('../../results/clustering/bert_embeddings_clustering_results.csv')

print(f"Results saved:")
print(f"- Parquet: results/clustering/bert_embeddings_clustering_results.parquet")
print(f"- CSV: results/clustering/bert_embeddings_clustering_results.csv")
print(f"\nResults shape: {results_df.shape}")
print(f"Columns: {results_df.columns}")

results_df.head()

## Summary

This notebook performed comprehensive clustering analysis on H&M product BERT embeddings:

### Data

- 42,229 articles with 768-dimensional BERT embeddings
- Used PCA (50 components) for computational efficiency
- Applied t-SNE and UMAP for 2D visualisation

### Clustering Methods

1. **K-means**: Determined optimal k using multiple metrics
2. **Hierarchical**: Ward linkage with dendrogram analysis
3. **HDBSCAN**: Density-based clustering for natural cluster detection

### Results

- Interactive visualisations saved to `results/clustering/`
- Clustering assignments and 2D coordinates saved for further analysis
- Performance metrics calculated for method comparison

The BERT embeddings enable semantic clustering of products based on textual descriptions, revealing natural product groupings that can support recommendation systems and inventory categorisation.
